# Blocher Reading Quiz
> Creating the reading quiz gradio app with state, reading list, vector store, chat, and chat download.

In this notebook, we add the reading list and vector store creating to the basic interaction app to be deployed onto Huggingface Space, which has the following components: 

- Readings:
    Display a list of readings uploaded by professor. 
- Student interface: 
    A chat interface where students interact with the chatbot that has had the instructor-designed prompt given to it but they cannot see the prompt. At the end of the conversation, entire convo is made available as downloadable JSON such that students can download it and turn it in to Brightspace.

In [1]:
#| default_exp ReadingQuiz

First, we'll start by loading our own libraries. Keep in mind that if you're on Colab, you need to replace "token" below with your GitHub token.

In [2]:
# run this code only if you're using Google Colab
#! pip install pip install git+https://ghusername:<token>@github.com/vanderbilt-data-science/lo-achievement.git

In [3]:
## run this code only if you're local and developing
import os, sys

# get parent of current working directory's parent (we're quite nested)
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# append to path
sys.path.append(parent_dir)

In [4]:
#| export
import os
import gradio as gr
import pandas as pd
from functools import partial
from ai_classroom_suite.MediaVectorStores import *
from ai_classroom_suite.UIBaseComponents import *

/workspaces/lo-achievement/ai_classroom_suite/UIBaseComponents.py:354: GradioUnusedKwargWarning: You have unused kwarg parameters in Box, please remove them: {'scale': 1}
  with gr.Box(elem_id="sources-container", scale=1):


## Interface Helpers and Functionality

In the next section, we'll create some helper functions to make sure we're able to create the interface we want, and that it behaves nicely. 

### Vector Store Functions

In this section, we define several functions related to creating vectore store from a folder of files and write the vector store to file.

- Default values:
    - The default folder path is "context_files/" 
    - The default output file name is "vector_store.txt"
- vector_store_file_exist()
    - Check if the output vector store file already exists in the folder. 
- get_filepathts_from_filder(fold_path)
    - This function get all the files' paths from a specified folder. 
- write_vector_store_to_file(out_file_name, vs_button)
    - This function creates the vector store of a given list of files and writes that into output file. 

In [5]:
#| to_script
#  default folder path
folder_path = "context_files"
#  default output file name
out_file_name = "vector_store.txt"

# Check if vector store file already exist on disk
def vector_store_file_exist():
    # Get all files in the folder
    files = os.listdir(folder_path)
    # Check if output file already exist in this folder
    return (out_file_name in files)

In [6]:
#| to_script
# Helper function to get all files' paths from a folder
# Return a list of file paths except for README.txt and vector_store.txt (if exist)
def get_filepaths_from_folder(folder_path):
    # Store the paths of files
    filepath_list = []
    
    # Check if the specified folder exists
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return filepath_list
    
    # Get all the files in the folder
    files = os.listdir(folder_path)
    
    for file_name in files:
        # Excluding README.txt and vector_store.txt
        if file_name != "README.txt" and file_name != "vector_store.txt":
            # Get the file path for each item
            file_path = os.path.join(folder_path, file_name)
            # Check if the item is a file and not a subdirectory
            if os.path.isfile(file_path):
                filepath_list.append(file_path)
    
    return filepath_list

In [7]:
#| to_script
# Helper function to write content of files in a folder to output file
def write_vector_store_to_file(out_file_name):
    # If vector_store.txt already exist, return nothing
    if vector_store_file_exist():
        return gr.File(value=out_file_name, visible=False)
 
    # Only try to create the vector store if vector_store.txt doesn't exist
    else:
        # Call the function to read files (excluding README.txt and vector_store.txt) pathes
        filepath_list = get_filepaths_from_folder(folder_path)
        # Extract the text out from files
        files_content = files_to_text(filepath_list, chunk_size=100, chunk_overlap=20)
        
        # Write the vector_store onto the output file
        with open(out_file_name, "w") as f:
            for i in range(len(files_content)):
                item = str(files_content[i]) + "\n"
                f.write(item)
        
        # Show the downlodable vector store file and give instruction on upload the vector store file to disk (on HuggingFace)
        return gr.File(title="Download your vector store file and upload it into the context_files folder under Files", 
                       value=out_file_name, visible=True)

# The User Interface

Below, we put all of this information together with the actual formatting of the user interface.

### Note for Instructors ###

1. Provide an OpenAI API Key
- To set the API Key, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``OPENAI_API_KEY`` value with your key.
- If you haven't created one already, visit [platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys) to sign up for an account and get your personal API key. 
2. Provide a Secret Prompt 
- To set the Secret Prompt, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``SECRET_PROMPT`` value with your prompt.
3. Upload files into vector store
- On Hugging Face Space, in ``Files`` Section, there's already a folder called ``context_files`` already created for you. 
- **Don't delete the file called ``README.txt``** as it is used to keep the ``context_files`` folder exist. You can change the content of that. It will not be read into the vector store. 
- You can delete or upload any other files you like into this ``context_files`` folder, and these will be read into the vector store.
- The final vector store file called ``vector_store.txt`` will be stored in this ``context_files`` folder as well. 

### Helper Function for Button

In [8]:
#| to_script
# To show the loading process on the button when creating vector store file
def creating_vs_button(obj_in):
    return gr.update(interactive=False, value='Creating Vector Store file...')
# To show the loading process on the button when initializing tutor
def initializing_tutor_button(obj_in):
    return gr.update(interactive=False, value='Initializing Tutor...')

### Helper function for Chatbot

In [9]:
#| to_script
# overwrites the original method since we don't deal with any vector stores display here
def get_tutor_reply(chat_tutor):
    chat_tutor.get_tutor_reply()
    return gr.update(value="", interactive=True), chat_tutor.conversation_memory, chat_tutor

def get_conversation_history(chat_tutor):
    return chat_tutor.conversation_memory, chat_tutor

### Actual Interface

In [10]:
#| to_script
with gr.Blocks() as ReadingQuiz:
    #initialize tutor (with state)
    study_tutor = gr.State(SlightlyDelusionalTutor())

    # Student chatbot interface
    gr.Markdown("""
    ## Chat with the Model
    Description here
    """)
    
    # Instead of ask students to provide key, the key is now provided by the instructor. 
    api_input = gr.Textbox(show_label=False, type="password", visible=False, value=os.environ.get("OPENAI_API_KEY"))

    # The instructor will provide a secret prompt/persona to the tutor
    instructor_prompt = gr.Textbox(label="Verify your prompt content", value = os.environ.get("SECRET_PROMPT"), visible=False)
    
    # Show input files
    file_input = gr.File(label="Reading materials", value=get_filepaths_from_folder(folder_path), visible=True)
    
    # Show output file for vector store when needed
    vs_file_name = gr.Text(visible=False, value=out_file_name)
    file_output = gr.File(visible=False)
    
    # Placeholders components
    text_input_none = gr.Textbox(visible=False)
    file_input_none = gr.File(visible=False)
    instructor_input_none = gr.TextArea(visible=False)
    learning_objectives_none = gr.Textbox(visible=False)

    # Set the secret prompt in this session and embed it to the study tutor
    vs_build_button = gr.Button("Initialize Tutor")
    vs_build_button.click(
        fn=creating_vs_button, inputs=vs_build_button, outputs=vs_build_button
    ).then(
        fn=write_vector_store_to_file, inputs=[vs_file_name], outputs=[file_output]
    ).then(
        fn=initializing_tutor_button, inputs=[vs_build_button], outputs=[vs_build_button]
    ).then(
        fn=create_reference_store, 
        inputs=[study_tutor, vs_build_button, instructor_prompt, file_output, instructor_input_none, api_input, learning_objectives_none],
        outputs=[study_tutor, vs_build_button]
    )

    with gr.Row(equal_height=True):
        with gr.Column(scale=2):
            chatbot = gr.Chatbot()
            with gr.Row():
                user_chat_input = gr.Textbox(label="User input", scale=9)
                user_chat_submit = gr.Button("Ask/answer model", scale=1)

    # First add user's message to the conversation history
    # Then get reply from the tutor and add that to the conversation history
    user_chat_submit.click(
        fn = add_user_message, inputs = [user_chat_input, study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=False
    ).then(
        fn = get_tutor_reply, inputs = [study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=True
    )
    # User can also press "Enter" on keyboard to submit a message
    user_chat_input.submit(
        fn = add_user_message, inputs = [user_chat_input, study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=False
    ).then(
        fn = get_tutor_reply, inputs = [study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=True
    )

    # Download conversation history file
    with gr.Blocks():
        gr.Markdown("""
        ## Export Your Chat History
        Export your chat history as a .json, .txt, or .csv file
        """)
        with gr.Row():
            export_dialogue_button_json = gr.Button("JSON")
            export_dialogue_button_txt = gr.Button("TXT")
            export_dialogue_button_csv = gr.Button("CSV")
            
        file_download = gr.Files(label="Download here", file_types=['.json', '.txt', '.csv'], type="file", visible=False)
    
    export_dialogue_button_json.click(save_json, study_tutor, file_download, show_progress=True)
    export_dialogue_button_txt.click(save_txt, study_tutor, file_download, show_progress=True)
    export_dialogue_button_csv.click(save_csv, study_tutor, file_download, show_progress=True)

In [11]:
ReadingQuiz.queue().launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/usr/local/lib/python3.10/dist-packages/gradio/helpers.py:818: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Textbox(...)` instead of `return gr.update(...)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/components/button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
/usr/lib/python3.10/asyncio/runners.py:44: GradioUnusedKwargWarning: You have unused kwarg parameters in File, please remove them: {'title': 'Download your vector store file and upload it into the context_files folder under Files'}
  return loop.run_until_complete(main)
/usr/local/lib/python3.10/dist-packages/gradio/helpers.py:818: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Textbox(...)` instead of `return gr.update(...)
  warnings.warn(
/usr/local/lib/p

I just got called.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 388, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 217, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1553, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1191, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

Keyboard interruption in main thread... closing server.


In [ ]:
#| to_script
ReadingQuiz.queue().launch(server_name='0.0.0.0', server_port=7860)

OSError: Cannot find empty port in range: 7860-7860. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.

A little helper in case your ports are open and you just want to close them all. If this doesn't work, restart your IDE.

In [ ]:
gr.close_all()